In [1]:
from datasets import load_dataset
from huggingface_hub import interpreter_login
import random
import json
import os
from dotenv import load_dotenv
import openai

load_dotenv("/Users/wongyenchik/Desktop/finetune llama/.env")
aws_access_key_id = os.getenv("AWS_ACCESS_KEY")
aws_secret_access_key = os.getenv("AWS_SECRET_KEY")
region_name = os.getenv("AWS_REGION")
openai_api_key = os.getenv("OPENAI_API_KEY")

# Set the OpenAI API key
openai.api_key = openai_api_key

import subprocess
import boto3
from mlx_lm import load, generate

interpreter_login()

/Users/wongyenchik/anaconda3/envs/finetune-model/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



In [2]:
huggingface_dataset_name = "HuggingFaceH4/MATH-500"
dataset = load_dataset(huggingface_dataset_name)


In [3]:
dataset['test'] = dataset['test'].remove_columns(['subject', 'level', 'unique_id'])

# Check the result to see if the 'input' column is removed
dataset['test'][0]


{'problem': 'Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\\theta),$ where $r > 0$ and $0 \\le \\theta < 2 \\pi.$',
 'solution': 'We have that $r = \\sqrt{0^2 + 3^2} = 3.$  Also, if we draw the line connecting the origin and $(0,3),$ this line makes an angle of $\\frac{\\pi}{2}$ with the positive $x$-axis.\n\n[asy]\nunitsize(0.8 cm);\n\ndraw((-0.5,0)--(3.5,0));\ndraw((0,-0.5)--(0,3.5));\ndraw(arc((0,0),3,0,90),red,Arrow(6));\n\ndot((0,3), red);\nlabel("$(0,3)$", (0,3), W);\ndot((3,0), red);\n[/asy]\n\nTherefore, the polar coordinates are $\\boxed{\\left( 3, \\frac{\\pi}{2} \\right)}.$',
 'answer': '\\left( 3, \\frac{\\pi}{2} \\right)'}

In [4]:

num_test = 100
num_val = 100

In [5]:
instruction_list = []
response_list = []

for line in dataset['test']:
    instruction_list.append(line['problem'])
    response_list.append(line['solution'] + " The answer is:" + line['answer'] + " -MathGPT")

In [6]:
# instruction_list[0]
response_list[1]

'We count the number of times $\\frac{1}{n^3}$ appears in the sum\n\\[\\sum_{j = 1}^\\infty \\sum_{k = 1}^\\infty \\frac{1}{(j + k)^3},\\]where $n$ is a fixed positive integer.  (In other words, we are conditioning the sum on $j + k$.)  We get a term of $\\frac{1}{n^3}$ each time $j + k = n.$  The pairs $(j,k)$ that work are $(1,n - 1),$ $(2,n - 2),$ $\\dots,$ $(n - 1,1),$ for a total of $n - 1$ pairs.  Therefore,\n\\begin{align*}\n\\sum_{j = 1}^\\infty \\sum_{k = 1}^\\infty \\frac{1}{(j + k)^3} &= \\sum_{n = 1}^\\infty \\frac{n - 1}{n^3} \\\\\n&= \\sum_{n = 1}^\\infty \\left( \\frac{n}{n^3} - \\frac{1}{n^3} \\right) \\\\\n&= \\sum_{n = 1}^\\infty \\left( \\frac{1}{n^2} - \\frac{1}{n^3} \\right) \\\\\n&= \\sum_{n = 1}^\\infty \\frac{1}{n^2} - \\sum_{n = 1}^\\infty \\frac{1}{n^3} \\\\\n&= \\boxed{p - q}.\n\\end{align*} The answer is:p - q -MathGPT'

In [7]:
# prompt format
intstructions_string = f"""MathGPT, functioning as a virtual Math chatbot, communicates in clear, accessible language, always provide the students steps to solve the question before provide answer. \
It reacts to feedback aptly and ends responses with its signature 'MathGPT'. \
MathGPT will tailor the length of its responses to match the student's questions, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Please respond to the following comment.
"""

example_template = lambda comment, response: f'''<s>[INST] {intstructions_string} \n{comment} \n[/INST]\n''' + response + "</s>"

example_list = []
for i in range(len(instruction_list)):
    example = {"text":example_template(instruction_list[i],response_list[i])}
    example_list.append(example)

print(example_list[-1])

{'text': '<s>[INST] MathGPT, functioning as a virtual Math chatbot, communicates in clear, accessible language, always provide the students steps to solve the question before provide answer. It reacts to feedback aptly and ends responses with its signature \'MathGPT\'. MathGPT will tailor the length of its responses to match the student\'s questions, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.\n\nPlease respond to the following comment.\n \nAltitudes $\\overline{AD}$ and $\\overline{BE}$ of $\\triangle ABC$ intersect at $H$.  If $\\angle BAC = 54^\\circ$ and $\\angle ABC = 52^\\circ$, then what is $\\angle AHB$? \n[/INST]\nFirst, we build a diagram:\n\n[asy]\n\nsize(150); defaultpen(linewidth(0.8));\n\npair B = (0,0), C = (3,0), A = (1.8,2), P = foot(A,B,C), Q = foot(B,A,C),H = intersectionpoint(B--Q,A--P);\n\ndraw(A--B--C--cycle);\n\ndraw(A--P^^B--Q);\n\nlabel("$A$",A,N); label("$B$",B,W); label("$

In [8]:
# create test and val data
test_val_index_list = random.sample(range(0, len(example_list)-1), num_test+num_val)

test_list = [example_list[index] for index in test_val_index_list[:num_test]]
val_list = [example_list[index] for index in test_val_index_list[num_test:]]

for example in test_list+val_list:
    example_list.remove(example)

Store train, test and valid data as jsonl as MLX model required

In [ ]:
os.makedirs('data', exist_ok=True)

with open('data/train.jsonl', 'w') as outfile:
    for example in example_list:
        json.dump(example, outfile)
        outfile.write('\n')

In [ ]:
with open('data/test.jsonl', 'w') as outfile:
    for example in test_list:
        json.dump(example, outfile)
        outfile.write('\n')

In [ ]:
with open('data/valid.jsonl', 'w') as outfile:
    for example in val_list:
        json.dump(example, outfile)
        outfile.write('\n')

Upload data to s3

In [ ]:

# Initialize the S3 client
s3 = boto3.client(
    service_name='s3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name
)

# Upload function
def upload_to_s3(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket inside a specific folder"""
    if object_name is None:
        object_name = file_name

    s3.upload_file(file_name, bucket, object_name)
    print(f"File '{file_name}' uploaded successfully to '{bucket}/{object_name}'")
    return True

# Local file path
file_names = ['data/train.jsonl', 'data/test.jsonl', 'data/valid.jsonl']

# S3 bucket name and S3 folder
bucket_name = 'finetune-model-layer'
s3_folder = 'data'  # This is the folder in the S3 bucket

for file_name in file_names:
    # Specify the full S3 object name (key)
    object_name = f"{s3_folder}/{file_name.split('/')[-1]}"

    # Upload the file to S3
    upload_to_s3(file_name, bucket_name, object_name)

Quantized model

In [ ]:
model = "google/gemma-2-2b-it"
command = [
    'python', '-m', 'mlx_lm.convert', '--hf-path', model , '-q'
]
subprocess.run(command)


Create lora layer

In [ ]:

# Define variables
model = "mlx_model"
iters = 10
steps_per_eval = 20
val_batches = -1
learning_rate = 1e-5
adapter_path = "adapter-2"


import subprocess
import mlflow
from datetime import datetime

# Define MLflow experiment
mlflow.set_experiment("MLX LoRA Fine-Tuning")

with mlflow.start_run(run_name=f"MLX-Model-LoRA-{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}"):

    # Log hyperparameters
    mlflow.log_param("model", model)
    mlflow.log_param("iters", iters)
    mlflow.log_param("steps_per_eval", steps_per_eval)
    mlflow.log_param("val_batches", val_batches)
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("adapter_path", adapter_path)

    # Create the command list
    command = [
        'python', '-m', 'mlx_lm.lora', '--model', model, '--train',
        '--iters', str(iters),
        '--steps-per-eval', str(steps_per_eval),
        '--val-batches', str(val_batches),
        '--learning-rate', str(learning_rate),
        '--test', 
        '--adapter-path', adapter_path
    ]
    
    # Run the subprocess command to execute fine-tuning
    subprocess.run(command)

Store lora layer at s3

In [ ]:
# Initialize the S3 client
s3 = boto3.client(
    service_name='s3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name
)

# Upload function
def upload_to_s3(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket inside a specific folder"""
    if object_name is None:
        object_name = file_name

    s3.upload_file(file_name, bucket, object_name)
    print(f"File '{file_name}' uploaded successfully to '{bucket}/{object_name}'")
    return True

# Local file path
folder_name = 'adapters'

# S3 bucket name and S3 folder
bucket_name = 'finetune-model-layer'
s3_folder = 'lora_layer'  # This is the folder in the S3 bucket

# Loop through all files in the adapters folder
for root, dirs, files in os.walk(folder_name):
    for file in files:
        # Get the full file path
        file_path = os.path.join(root, file)
        
        # Create the object name by preserving the folder structure
        object_name = os.path.join(s3_folder, os.path.relpath(file_path, folder_name)).replace("\\", "/")
        
        # Upload the file to S3
        upload_to_s3(file_path, bucket_name, object_name)


Test the ai (human evaluation)

In [16]:
# prompt format
intstructions_string = f"""MathGPT, functioning as a virtual Math chatbot, communicates in clear, accessible language, always provide the students steps to solve the question before provide answer. \
It reacts to feedback aptly and ends responses with its signature 'MathGPT'. \
MathGPT will tailor the length of its responses to match the student's questions, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Please respond to the following comment.
"""

prompt_template = lambda comment: f'''<s>[INST] {intstructions_string} \n{comment} \n[/INST]\n'''


Finetuned model

In [ ]:

# Define variables
comment = "In 1992, a scoop of gelato could be purchased in Italy for 1200 lire. The same gelato would have cost $\\$1.50$ in the U.S. At the equivalent exchange rate between the lire and the dollar, how many dollars would be equivalent to 1,000,000 lire?"

# Generate the prompt using the comment
prompt = prompt_template(comment)

# Create the command list
command = [
    'python', '-m', 'mlx_lm.generate',
    '--model', 'mlx_model',
    '--adapter-path', 'adapters',
    '--max-tokens', '512',
    '--prompt', prompt
]

# Run the command
subprocess.run(command)


Without finetune

In [ ]:
# Define variables
comment = "In 1992, a scoop of gelato could be purchased in Italy for 1200 lire. The same gelato would have cost $\\$1.50$ in the U.S. At the equivalent exchange rate between the lire and the dollar, how many dollars would be equivalent to 1,000,000 lire?"

# Generate the prompt using the comment
prompt = prompt_template(comment)

# Create the command list
command = [
    'python', '-m', 'mlx_lm.generate',
    '--model', 'mlx_model',
    '--max-tokens', '512',
    '--prompt', prompt
]

# Run the command
subprocess.run(command)

Test the response with openai and store in MLflow

Create dataset to test

In [19]:
import pandas as pd

# Create the test dataset (without the response)
inputs = []
ground_truth = []

# Go through each example in test_list
for example in test_list:
    # Extract the comment part from the 'text' field (everything between [INST] and [/INST])
    inst_start = example['text'].find("[INST]") + len("[INST]")
    inst_end = example['text'].find("[/INST]")

    # Extract the comment (instruction) and the response
    questions = example['text'][inst_start:inst_end].strip()  # This is the prompt for the test data
    response = example['text'][inst_end + len("[/INST]"):-4].strip()  # This is the corresponding answer

    # Append to inputs and ground truth
    inputs.append(questions)
    ground_truth.append(response)

# Convert inputs and ground_truth to a DataFrame
eval_data = pd.DataFrame(
    {
        "inputs": inputs,        
        "ground_truth": ground_truth
    }
)

# Example to see the eval_data
eval_data[:10]

,inputs,ground_truth
0,"MathGPT, functioning as a virtual Math chatbot...","By the associative property of multiplication,..."
1,"MathGPT, functioning as a virtual Math chatbot...","Since $10^x - 10 = 9990,$ we have $$10^x = 999..."
2,"MathGPT, functioning as a virtual Math chatbot...","Squaring both sides, we get\n\[x + \sqrt{3x + ..."
3,"MathGPT, functioning as a virtual Math chatbot...","Setting $x = 0,$ we get $|c| \le 1.$ Setting ..."
4,"MathGPT, functioning as a virtual Math chatbot...",Let $\mathbf{v} = \begin{pmatrix} a \\ b \\ c ...
5,"MathGPT, functioning as a virtual Math chatbot...",We count the number of times $\frac{1}{n^3}$ a...
6,"MathGPT, functioning as a virtual Math chatbot...",We can write\n\begin{align*}\n\frac{1}{\cos^2 ...
7,"MathGPT, functioning as a virtual Math chatbot...",Let $s_1 = x + y + z$ and $s_2 = xy + xz + yz....
8,"MathGPT, functioning as a virtual Math chatbot...",The hexagon has a side length of 16 centimeter...
9,"MathGPT, functioning as a virtual Math chatbot...",Suppose that $\angle ADC = x^\circ$. The area ...


Create the structure of MLX model in MLFlow

In [20]:
import mlflow.pyfunc
import subprocess

class MLXModelWrapper(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        # Store the model path, which will be used later in the prediction step
        self.model_path = context.artifacts["model"]

    def predict(self, context, model_input):
        # Log the input type
        print(f"Input type: {type(model_input)}")
        print(f"Input data: {model_input}")

        # Check if the input is a DataFrame and extract the 'inputs' column
        if isinstance(model_input, pd.DataFrame):
            if 'inputs' in model_input.columns:
                # Loop through the DataFrame and process each row
                responses = []
                for comment in model_input['inputs']:
                    prompt = self.prompt_template(comment)

                    command = [
                        'python', '-m', 'mlx_lm.generate',
                        '--model', self.model_path,  # Use the stored model path from load_context
                        '--max-tokens', '512',
                        '--prompt', prompt
                    ]

                    # Run the command to generate the response
                    result = subprocess.run(command, capture_output=True, text=True)
                    responses.append(result.stdout)
                return responses
            else:
                raise ValueError("Expected 'inputs' column in the DataFrame.")
        else:
            raise ValueError(f"Unexpected input type: {type(model_input)}")

    def prompt_template(self, comment):
        # Custom prompt template
        instructions_string = """MathGPT, functioning as a virtual Math chatbot, communicates in clear, accessible language, always providing the students steps to solve the question before providing the answer. \
It reacts to feedback aptly and ends responses with its signature 'MathGPT'. \
MathGPT will tailor the length of its responses to match the student's questions, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.
Please respond to the following comment.
"""
        return f'''<s>[INST] {instructions_string} \n{comment} \n[/INST]\n'''


/Users/wongyenchik/anaconda3/envs/finetune-model/lib/python3.13/site-packages/mlflow/pyfunc/utils/data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


Log the model in MLFlow and evaluate the model

In [21]:
from mlflow.metrics import latency
from mlflow.metrics.genai import answer_correctness
import time
import mlflow
signature = mlflow.models.infer_signature(
    model_input=pd.DataFrame({
    'inputs': [
        "[INST] MathGPT, functioning as a virtual Math chatbot, communicates in clear, accessible language, always providing steps before the answer. ...",
    ]
}),
    model_output=["==========\nLet's solve that!  \n\nTo find the answer, we simply add the numbers together: 4 + 4 = 8. \n\nSo, 4 + 4 = 8. \n\nMathGPT \n\n==========\nPrompt: 119 tokens, 102.358 tokens-per-sec\nGeneration: 49 tokens, 15.268 tokens-per-sec\nPeak memory: 1.713 GB\n",
 "==========\nTo find the derivative of x², we use the power rule of differentiation.  \n\nThe power rule states that the derivative of x<sup>n</sup> is nx<sup>n-1</sup>.\n\nApplying this to x²:\n\n*  The derivative of x² is 2x<sup>2-1</sup> \n*  Simplifying, we get 2x<sup>1</sup> \n*  Therefore, the derivative of x² is **2x**. \n\n\nMathGPT \n\n==========\nPrompt: 120 tokens, 118.289 tokens-per-sec\nGeneration: 99 tokens, 18.342 tokens-per-sec\nPeak memory: 1.714 GB\n"],
)
input_example = pd.DataFrame({
    'inputs': [
        "[INST] MathGPT, functioning as a virtual Math chatbot, communicates in clear, accessible language, always providing steps before the answer. ...",
    ]
})

# The evaluation process remains the same
mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment("LLM Evaluation")

with mlflow.start_run() as run:
    logged_model_info = mlflow.pyfunc.log_model(
        artifact_path="model",  # Path where the model will be stored in MLflow
        python_model=MLXModelWrapper(),  # Your custom model wrapper
        artifacts={"model": "/Users/wongyenchik/Desktop/finetune llama/my-model"},  # The path to your saved MLX model
        signature=signature,
        input_example=input_example
    )

    # Use the function to evaluate the model
    results = mlflow.evaluate(
        logged_model_info.model_uri,
        data=eval_data[:10],
        targets="ground_truth",
        model_type="question-answering",
        extra_metrics=[
            answer_correctness(),
            latency(),
        ],
    )

    # Example output DataFrame for inspection
    print(results)


2025/03/05 22:29:07 INFO mlflow.tracking.fluent: Experiment with name 'LLM Evaluation' does not exist. Creating a new experiment.
2025/03/05 22:29:07 INFO mlflow.pyfunc: Validating input example against model signature


Input type: <class 'pandas.core.frame.DataFrame'>
Input data:                                               inputs
0  [INST] MathGPT, functioning as a virtual Math ...


2025/03/05 22:30:15 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.
2025/03/05 22:30:15 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.


Input type: <class 'pandas.core.frame.DataFrame'>
Input data:                                               inputs
0  MathGPT, functioning as a virtual Math chatbot...
Input type: <class 'pandas.core.frame.DataFrame'>
Input data:                                               inputs
1  MathGPT, functioning as a virtual Math chatbot...
Input type: <class 'pandas.core.frame.DataFrame'>
Input data:                                               inputs
2  MathGPT, functioning as a virtual Math chatbot...
Input type: <class 'pandas.core.frame.DataFrame'>
Input data:                                               inputs
3  MathGPT, functioning as a virtual Math chatbot...
Input type: <class 'pandas.core.frame.DataFrame'>
Input data:                                               inputs
4  MathGPT, functioning as a virtual Math chatbot...
Input type: <class 'pandas.core.frame.DataFrame'>
Input data:                                               inputs
5  MathGPT, functioning as a virtual Math cha

2025/03/05 22:34:42 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
Device set to use mps:0
100%|██████████| 10/10 [00:09<00:00,  1.07it/s]

🏃 View run mysterious-midge-856 at: http://127.0.0.1:5000/#/experiments/880270386239245598/runs/963ef29d73b34430959321a914b68a5e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/880270386239245598


Save model and push to HuggingFace

In [ ]:
# Define variables
save_path = "my-model"

# Create the command list
command = [
    'python', '-m', 'mlx_lm.fuse',
    '--model', 'mlx_model',
    '--save-path', save_path,
    '--adapter-path', 'adapters',
    '--upload-repo', 'yenchik/mlx-gemma-2-2b-it-math',
    '--hf-path', 'google/gemma-2-2b-it'
]

# Run the command
subprocess.run(command)
    

Access model from HuggingFace

In [ ]:
model, tokenizer = load("yenchik/mlx-gemma-2-2b-it-math")

prompt = "In 1992, a scoop of gelato could be purchased in Italy for 1200 lire. The same gelato would have cost $\\$1.50$ in the U.S. At the equivalent exchange rate between the lire and the dollar, how many dollars would be equivalent to 1,000,000 lire?"
if tokenizer.chat_template is not None:
    messages = [{"role": "user", "content": prompt}]
    prompt = tokenizer.apply_chat_template(
        messages, add_generation_prompt=True
    )

response = generate(model, tokenizer, prompt=prompt, verbose=True)
